In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from getchinamap.getchinamap import DownloadChmap
import rasterio.mask
import cartopy.crs as ccrs
import matplotlib as mpl
import datetime
dl_engine = DownloadChmap()



In [ ]:

dem_raw_data = rasterio.open("数据集/ChinaDEM1km_WGS84.tif")
# dem_raw_data = rasterio.open("数据集/Global_DEM/World_e-Atlas-UCSD_SRTM30-plus_v8.tif")


In [ ]:
dem_raw_data.crs

In [ ]:
dem_raw_data.bounds

In [ ]:
dem_raw_data.transform

In [ ]:
%matplotlib

In [ ]:
# china_boundary_gdf = dl_engine.download_country(target='边界')
province_name = '西藏自治区'
china_boundary_gdf = dl_engine.download_province(province_name=province_name,target='边界')
china_boundary_gdf_sub = dl_engine.download_province(province_name=province_name,target='市')
# china_boundary_gdf = dl_engine.download_country(target='边界')
# china_boundary_gdf_sub = dl_engine.download_country(target='省')
china_boundary_gdf_sub

In [ ]:
out_image, out_transform = rasterio.mask.mask(dem_raw_data, china_boundary_gdf['geometry'], crop=True)
out_meta = dem_raw_data.meta
out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

In [ ]:
clean_data = out_image.copy().astype('float')
clean_data = clean_data[0, :, :]
clean_data[clean_data == out_meta['nodata']] = np.nan
# clean_data[clean_data <= -200] = np.nan

# clean_data

## 全部都是ccrs.PlateCarree()

In [ ]:
mat_extent = [getattr(out_transform, 'c'),
              getattr(out_transform, 'c') + clean_data.shape[1] * getattr(out_transform, 'a'),
              getattr(out_transform, 'f') + clean_data.shape[0] * getattr(out_transform, 'e'),
              getattr(out_transform, 'f')]

colors = ["#33A02C", "#B2DF8A", "#FDBF6F", "#1F78B4", "#999999", "#E31A1C", "#E6E6E6", "#A6CEE3"]

myccrs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=(10, 5), dpi=150, subplot_kw={'projection': myccrs})

ax.gridlines(draw_labels=True,
             linewidth=2, color='gray', alpha=0.5, linestyle='--')

ax_im_bar = ax.imshow(clean_data, origin='upper', extent=mat_extent, transform=myccrs,
                      cmap=mpl.colors.LinearSegmentedColormap.from_list("mypalette", colors, N=1000))

china_boundary_gdf_sub.boundary.plot(ax=ax, color='black', linewidth=1)

fig.colorbar(ax_im_bar, orientation='vertical')
# plt.tight_layout()
#
# ax.set_title("china dem")
# ax.set_ylabel("y label")
# ax.set_xlabel("x label")

In [ ]:
np.nanmin(clean_data), np.nanmax(clean_data)

## 别的投影

In [ ]:
longitude = np.array(
    [getattr(out_transform, 'c') + i * getattr(out_transform, 'a') for i in range(clean_data.shape[1])])
latitude = np.array(
    [getattr(out_transform, 'f') + i * getattr(out_transform, 'e') for i in range(clean_data.shape[0])])

colors = ["#33A02C", "#B2DF8A", "#FDBF6F", "#1F78B4", "#999999", "#E31A1C", "#E6E6E6", "#A6CEE3"]
center_china_point = china_boundary_gdf.centroid[0]

# 新的兰伯特投影
new_crs = ccrs.Orthographic(central_longitude=center_china_point.x,
                                central_latitude=center_china_point.y)
myccrs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=(10, 10), dpi=100, subplot_kw={'projection': new_crs})

ax.gridlines(draw_labels=True,
             linewidth=2, color='gray', alpha=0.5, linestyle='--')
ax_im_bar = ax.contourf(longitude, latitude, clean_data,  # [::-1,:]
                            np.arange(0, np.nanmax(clean_data), 100.0),
                            transform=ccrs.PlateCarree(),
                            cmap=mpl.colors.LinearSegmentedColormap.from_list("mypalette", colors, N=1000),
                            extend='both', origin='upper')
china_boundary_gdf_sub.to_crs(new_crs.proj4_init).boundary.plot(ax=ax, edgecolor='black', linewidth=1)
fig.colorbar(ax_im_bar, orientation='vertical')
# fig.savefig('test1226.png')
fig.savefig(f"test1226_china_{datetime.datetime.now().strftime('%Y-%m-%d %X')}.png")